In [19]:
# import statements
import requests
import re
import json
from bs4 import BeautifulSoup as BS

In [20]:
# grab page
url = "https://en.wikipedia.org/wiki/1909_Giro_d'Italia"

# a header is needed to scrape data from wikipedia
headers = {
    'User-Agent': 'SchoolProjectBot/1.0 (r1043412@student.thomasmore.be) Python-Requests/2.31.0'
}

page = requests.get(url, headers=headers)
soup = BS(page.text, "html")

In [21]:
# table with all information
table = soup.find('table', class_ = 'wikitable')

In [22]:
# use list comprehension instead of for loop
courses = [
    # take all td's and put them in a list 'cells', take the second and third elements from that list
    (lambda cells: (cells[1].get_text(strip=True), cells[2].get_text(strip=True)))(row.find_all("td"))
    # take all rows exept title and results row
    for row in table.select('tr')[1:-1]
]

print(courses)

[('MilantoBologna', '397\xa0km (247\xa0mi)'), ('BolognatoChieti', '375.8\xa0km (233.5\xa0mi)'), ('ChietitoNaples', '242.8\xa0km (150.9\xa0mi)'), ('NaplestoRome', '228.1\xa0km (141.7\xa0mi)'), ('RometoFlorence', '346.5\xa0km (215.3\xa0mi)'), ('FlorencetoGenoa', '294.1\xa0km (182.7\xa0mi)'), ('GenoatoTurin', '354.9\xa0km (220.5\xa0mi)'), ('TurintoMilan', '206\xa0km (128\xa0mi)')]


In [23]:
# clean data
giro_1909 = dict()
courses = [
    (lambda course:
    (course[0][:re.search('to[A-Z]', course[0]).start()],
    course[0][re.search('to[A-Z]', course[0]).end()-1:],
    course[1][:re.search(r'\D', course[1]).start()])
    )(course)
    for course in courses
]

print(courses)

[('Milan', 'Bologna', '397'), ('Bologna', 'Chieti', '375'), ('Chieti', 'Naples', '242'), ('Naples', 'Rome', '228'), ('Rome', 'Florence', '346'), ('Florence', 'Genoa', '294'), ('Genoa', 'Turin', '354'), ('Turin', 'Milan', '206')]


In [24]:
# print to json file
giro_1909 = [
    {'key': i, 'start': course[0], 'end': course[1], 'distance': course[2]}
    for i, course in enumerate(courses, start=1)
]

print(giro_1909)

with open('data/giro_ditalia.json', 'r+') as file:
    file_data = json.load(file)
    file_data['giro_ditalia'].update({1909: giro_1909})
    file.seek(0)
    json.dump(file_data, file, indent=4)

[{'key': 1, 'start': 'Milan', 'end': 'Bologna', 'distance': '397'}, {'key': 2, 'start': 'Bologna', 'end': 'Chieti', 'distance': '375'}, {'key': 3, 'start': 'Chieti', 'end': 'Naples', 'distance': '242'}, {'key': 4, 'start': 'Naples', 'end': 'Rome', 'distance': '228'}, {'key': 5, 'start': 'Rome', 'end': 'Florence', 'distance': '346'}, {'key': 6, 'start': 'Florence', 'end': 'Genoa', 'distance': '294'}, {'key': 7, 'start': 'Genoa', 'end': 'Turin', 'distance': '354'}, {'key': 8, 'start': 'Turin', 'end': 'Milan', 'distance': '206'}]
